# RAG: Поиск по файлам (File Search)

В этом ноутбуке мы создадим **персональную базу знаний** из научных статей и сделаем агента для работы с ними.

В конце — добавим интеграцию с **Telegram-ботом** для загрузки файлов и общения.

## Архитектура

```
┌─────────────────┐      ┌─────────────────┐      ┌─────────────────┐
│  Telegram Bot   │──────│   Agent +       │──────│  Vector Store   │
│  (pyTelegramBot)│      │   Responses API │      │  (RAG index)    │
└─────────────────┘      └─────────────────┘      └─────────────────┘
        │                                                │
        │                                                │
   Загрузка файлов  ◄────────────────────────────────────┘
```

Вначале установим необходимые библиотеки:

In [ ]:
%pip install --quiet openai python-dotenv pyTelegramBotAPI shwarsutils

> **ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

## Авторизация

Для работы с Yandex Cloud нам понадобится `folder_id` (идентификатор каталога) и `api_key` (API-ключ сервисного аккаунта). Мы предполагаем, что эти значения хранятся в файле `.env` в текущей директории, который можно скачать, выполнив ячейку ниже (убедитесь предварительно, что в ней указан правильный адрес для скачивания):

In [ ]:
!curl -o .env {{url_of_dotenv_file}}\n

In [14]:
import os
from glob import glob
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

def printx(string):
    display(Markdown(string))

print(f"✅ Авторизация настроена (folder_id: {folder_id[:8]}...)")

✅ Авторизация настроена (folder_id: b1gkp4q6...)


---

## Часть 1: Импорт класса Agent и создание клиента

В этом примере мы будем использовать класс `Agent`, который мы рассмотрели в предыдущем разделе применительно к вызову инструментов. Класс Agent позволяет указать список инструментов, которые сможет использовать агент, в виде Pydantic-классов (для локальных инструментов), или же в виде JSON-описания, которое используется для задания внутренних инструментов из Responses API.

Импортируем этот класс из файла [Agent.py](Agent.py):

In [5]:
from Agent import Agent, create_client

client = create_client()
print("✅ Клиент создан")

✅ Клиент создан


---

## Часть 2: Загрузка файлов в облако

В качестве примера мы будем делать свою базу знаний по научным статьям. Для примера возьмём пару научных статей в формате PDF и/или TXT. Вы можете загрузить статьи, выполнив ячейку ниже, или поместить свою библиотеку статей в директорию `data`:

In [6]:
from shwars.utils import *
download_file("https://raw.githubusercontent.com/yandex-ai-studio/ai-studio-course/main/data/papers.zip")
unzip_file("papers.zip")
print(" ✅ Данные загружены и распакованы.")

 ✅ Данные загружены и распакованы.



Посмотрим на все научные статьи из папки `data/`:

In [7]:
# Найдём файлы для загрузки
data_files = glob("data/*.pdf") + glob("data/*.txt")
print(f"📄 Найдено файлов: {len(data_files)}")
for f in data_files:
    print(f"   - {os.path.basename(f)}")

📄 Найдено файлов: 3
   - article-attention.pdf
   - article-prompteng.pdf
   - article-attention.txt


Прежде, чем добавлять файлы в поисковое хранилище, нам необходимо загрузить их в облако с помощью `client.files.create`:

In [8]:
# Загружаем файлы в Yandex Cloud
uploaded_files = []

for filepath in data_files:
    filename = os.path.basename(filepath)
    print(f"⬆️  Загружаем: {filename}")
    
    with open(filepath, "rb") as f:
        uploaded = client.files.create(
            file=f,
            purpose="assistants"
        )
    uploaded_files.append(uploaded)
    print(f"   ✓ ID: {uploaded.id}")

print(f"\n✅ Загружено файлов: {len(uploaded_files)}")

⬆️  Загружаем: article-attention.pdf
   ✓ ID: fvt3l7a8aq53lpr55f4r
⬆️  Загружаем: article-prompteng.pdf
   ✓ ID: fvt85d4bbhuc7h5hp8et
⬆️  Загружаем: article-attention.txt
   ✓ ID: fvt34ibgg34a40atbqld

✅ Загружено файлов: 3


Мы сохранили список всех загруженный файлов вместе с их идентификаторами в `uploaded_files` - они понадобятся нам для добавления в векторное хранилище.

---

## Часть 3: Создание векторного хранилища (Vector Store)

Vector Store — это индекс для семантического поиска по файлам. Сначала создаём само хранилище:

In [9]:
# Создаём векторное хранилище
vector_store = client.vector_stores.create(name="papers_knowledge_base")
print(f"📚 Создано хранилище: {vector_store.id}")

📚 Создано хранилище: fvti6gqjnvsmoslkt76b


Теперь перебираем все файлы и добавляем их в хранилище с помощью `client.vector_stores.files.create`. При этом происходит следующее:
- Файл преобразуется в текстовый формат. Хотя мы загружали файлы в формате PDF, для работы с LLM требуется текст.
- В случае необходимости длинные файлы подвергаются **чанкованию** - разбиению на более мелкие фрагменты, с небольшим перекрытием. Параметры чанкования мы указываем в параметре `chunking_strategy`
- Все чанки ставятся в очередь на индексирование

Индексирование как правило занимает не очень много времени, однако если объем файлов большой - времени может потребоваться больше. Если необходимо, можно проверить готовность индексации, но мы ради простоты кода этого делать не будем.

In [10]:
# Добавляем файлы в хранилище
for uploaded in uploaded_files:
    print(f"📥 Индексируем: {uploaded.filename}")
    client.vector_stores.files.create(
        vector_store_id=vector_store.id,
        file_id=uploaded.id,
        chunking_strategy={
            "type": "static",
            "static": {
                "max_chunk_size_tokens": 800,
                "chunk_overlap_tokens": 100
            }
        }
    )

print(f"\n✅ Добавление в индекс завершено")

📥 Индексируем: article-attention.pdf
📥 Индексируем: article-prompteng.pdf
📥 Индексируем: article-attention.txt

✅ Добавление в индекс завершено


---

## Часть 4: Проверка семантического поиска

Теперь, когда мы проиндексировали все документы, мы можем осуществлять поиск в векторном хранилище и находить документы, наиболее релевантные запросу:

In [12]:
results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="What is attention mechanism in neural networks?",
    rewrite_query=True
)

print("🔍 Результаты поиска:")
for r in results.data[:3]:
    print(f"   - {r.filename} (релевантность: {r.score:.3f})")
    printx(f"     {r.content[0].text[:150]}...")

🔍 Результаты поиска:
   - article-attention.pdf (релевантность: 0.500)


     
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The ...

   - article-attention.txt (релевантность: 0.500)


     
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The ...

   - article-attention.pdf (релевантность: 0.500)


     

$Attention (Q , K , V) = \operatorname {softmax} (\frac {QK ^ {T}} {\sqrt {d _ {k}}}) V$

(1)

The two most commonly used attention functions are ad...

Обратите внимание на параметр `rewrite_query` - он позволяет осуществить дополнительную перефразировку вопроса, чтобы повысить разнообразие найденных материалов.

Поиск по умолчанию осуществляется по **гибридному индексу**, включающему в себя:
* **Векторный поиск** на основе эмбеддинг-векторов
* **Поиск по ключевым словам**

Сочетание двух типов поиска позволяет наилучшим образом находить как фрагменты, связанные с конкретными ключевыми словами, так и фрагменты, подходящие по смыслу.  

---

## Часть 5: Создание агента с File Search

Интегрировать файловый поиск в агента (и в Responses API) на самом деле очень просто - достаточно описать инструмент **файлового поиска** в виде простого словаря, и передать его в список используемых инструментов:

In [39]:
# Инструмент для поиска по файлам
file_search_tool = {
    "type": "file_search",
    "vector_store_ids": [vector_store.id],
    "max_num_results": 5
}

res = client.responses.create(
    model = model,
    instructions = """Ты - научный редактор. Используй инструмент файлового 
    поиска для поиска по базе статей для ответа на вопросы пользователей.""",
    tools = [ file_search_tool ],
    input = "Расскажи на основе статей, что такое механизм внимания"
)

printx(res.output_text)

Механизм внимания (attention mechanism) — это ключевой компонент в архитектурах нейронных сетей, особенно в трансформерах, который позволяет модели фокусироваться на наиболее релевантных частях входной последовательности при обработке каждого элемента. Вместо того чтобы обрабатывать всю последовательность с равным весом, механизм внимания вычисляет веса (степень важности) для каждого элемента последовательности относительно текущего.

На основе представленных статей:

1. **Структура механизма внимания**:  
   Механизм использует три компонента — **Q (запрос, Query)**, **K (ключ, Key)** и **V (значение, Value)**. Сначала вычисляется схожесть между Q и K (обычно через скалярное произведение), затем применяется масштабирование (Scale) и функция SoftMax для получения весов внимания. Эти веса используются для взвешивания V, что даёт выход — взвешенную сумму значений. Эта операция реализуется через матричные умножения (MatMul), что отражено в блок-схеме.

2. **Опциональная маска (Mask)**:  
   В некоторых случаях (например, при обработке последовательностей разной длины или при обучении языковых моделей) используется маска, чтобы предотвратить внимание к определённым позициям (например, будущим словам). Это особенно важно в декодерах трансформеров.

3. **Визуализация внимания**:  
   На рисунках показано, как механизм внимания в реальных моделях может отслеживать долгосрочные зависимости (например, связь между словом "making" и "more difficult") или выполнять разрешение анафор (например, определение, на что указывает местоимение "its"). Разные "головки" (heads) внимания в многоголовой архитектуре (Multi-Head Attention) могут фокусироваться на разных типах зависимостей.

4. **Многоголовое внимание (Multi-Head Attention)**:  
   Чтобы избежать усреднения информации и повысить выразительность модели, используется несколько независимых механизмов внимания (головок), чьи выходы объединяются. Это позволяет модели одновременно учитывать различные аспекты контекста.

Таким образом, механизм внимания — это динамический способ взвешивания информации из входной последовательности, который значительно улучшает способность моделей обрабатывать контекст и зависимости на больших расстояниях.

Мы можем также посмотреть, из каких источников был получен этот ответ. Структура ответа модели очень похожа на веб-поиск:

In [18]:
print(f"Длина ответа: {len(res.output)}")
for a in res.output:
    print(f" - {a.type}")

Длина ответа: 2
 - file_search_call
 - message


Посмотрим на список всех найденных источников:

In [24]:
print(f"Запросы: {res.output[0].queries}")
for x in res.output[0].results:
    print(f" - file: {x.filename}, text = {x.text[:30]}...")

Запросы: ['механизм внимания']
 - file: article-attention.pdf, text =  - принимает выход Mask (opt.)...
 - file: article-attention.pdf, text = 

12



<PAGE/>

13

*`<CAPTIO...
 - file: article-attention.pdf, text = 

Attention Visualizations



...
 - file: article-attention.pdf, text =  due to averaging attention-we...
 - file: article-attention.txt, text =  due to averaging attention-we...


Также источники доступны и в ответе LLM (`res.output[1]`):

In [30]:
for x in res.output[1].content[0].annotations:
    print(f"{x.filename}, type={x.type}, index={x.index}")

article-attention.pdf, type=file_citation, index=0
article-attention.pdf, type=file_citation, index=0
article-attention.pdf, type=file_citation, index=0
article-attention.pdf, type=file_citation, index=0
article-attention.txt, type=file_citation, index=0


Интегрируем возможность файлового поиска в нашего ассистента:

In [43]:

# Промпт для научного ассистента
paper_assistant_instruction = """
Ты — научный ассистент, эксперт по анализу научных статей.

Твоя база знаний содержит научные публикации.

При ответе на вопросы:
1. Всегда используй инструмент файлового поиска, делай запросы на английском языке
2. Цитируй релевантные фрагменты из статей
3. Указывай источник информации
4. Объясняй сложные концепции простым языком

Если информации нет в базе знаний — честно об этом скажи.

Отвечай на русском языке.
"""

paper_agent = Agent(
    client=client,
    instruction=paper_assistant_instruction,
    tools=[file_search_tool],
    tool_choice="required"
)

print("📖 Научный ассистент готов!")

📖 Научный ассистент готов!


В ассистенте мы используем более проработанный системный промпт для повышения качества ответа.

Зададим тот же вопрос:

In [44]:
result = paper_agent("Что такое механизм внимания (attention) и как он работает?")
print(f"Запрос: {result.output[0].queries}")
print(f"Файлы: {set(x.filename for x in result.output[0].results)}")
printx(result.output_text)

Запрос: ['attention mechanism in machine learning']
Файлы: {'article-attention.pdf', 'article-attention.txt'}


**Механизм внимания (attention)** в машинном обучении — это метод, который позволяет модели учитывать взаимосвязи между различными частями входных данных при их обработке. Например, в обработке естественного языка механизм внимания помогает модели понять, какие слова или части предложения наиболее важны для выполнения конкретной задачи.

**Как работает механизм внимания:**

1. **Самовнимание (self-attention или intra-attention)** позволяет модели учитывать связи между разными позициями в одной последовательности данных. Это помогает вычислить представление всей последовательности, учитывая взаимосвязи между её элементами. Например, самовнимание успешно применяется для:
   - понимания текста (reading comprehension);
   - создания сжатых версий текста (abstractive summarization);
   - определения логических следствий из текста (textual entailment);
   - изучения независимых от задачи представлений предложений [4, 27, 28, 22].

2. **Существуют разные виды функций внимания:**
   - **Аддитивное внимание (additive attention)** использует нейронную сеть с одним скрытым слоем для вычисления функции совместимости.
   - **Скалярное (точечное) внимание (dot-product или multiplicative attention)** быстрее и эффективнее с точки зрения использования памяти. Оно основано на операции умножения матриц и включает масштабирующий фактор $\frac{1}{\sqrt{d_k}}$.

3. **Механизм внимания может улучшить интерпретируемость моделей.** Исследования показали, что отдельные «головы» внимания (attention heads) в моделях обучаются выполнять разные задачи и могут отражать синтаксическую и семантическую структуру предложений.

**Применение:**
- End-to-end memory networks (сети с памятью) используют рекуррентный механизм внимания вместо последовательного повторения и хорошо справляются с задачами, связанными с ответами на вопросы и моделированием языка [34].

**Источник:** научная публикация о механизмах внимания в машинном обучении.

А теперь зададим вопрос по другой статье:

In [45]:
result = paper_agent("Какие есть техники промпт-инжиниринга?")
print(f"Запрос: {result.output[0].queries}")
print(f"Файлы: {set(x.filename for x in result.output[0].results)}")
printx(result.output_text)

Запрос: ['prompt engineering techniques']
Файлы: {'article-prompteng.pdf'}


В промпт-инжиниринге существует несколько техник, которые помогают улучшить качество ответов больших языковых моделей (LLM). Вот некоторые из них:

1. **Conciseness and Clarity (лаконичность и ясность)**:
   - Промпты должны быть краткими и чёткими, без избыточной информации, которая может запутать модель или привести к нерелевантным ответам.
   - Цитата: «Generally, overly verbose or ambiguous prompts can confuse the model or lead to irrelevant responses. Thus, the prompt should be concise, avoiding unnecessary information that does not contribute to the task while being specific enough to guide the model».

2. **Ask-Me-Anything (техника „спрашивай меня о чём угодно“)**:
   - Использует несколько неидеальных промптов и агрегирует их результаты для улучшения производительности модели, особенно в формате ответов на вопросы.

3. **Chain-of-Thought (цепочка мыслей)**:
   - Модель генерирует серию промежуточных шагов рассуждения, что улучшает её работу над сложными задачами.

4. **Least-to-Most Prompting (от простого к сложному)**:
   - Стратегия разбиения сложных проблем на более простые подзадачи, что значительно улучшает способность модели решать более сложные задачи.

5. **Directional Stimulus Prompting (направляющий стимул)**:
   - Использует настраиваемую модель политики для генерации вспомогательных промптов, направляя LLM к конкретным желаемым результатам.

6. **Использование программистской логики**:
   - Включение в промпт условных операторов, логических операторов или псевдокода для направления процесса рассуждения модели.

Эти техники подчёркивают быстро развивающуюся область промпт-инжиниринга и предлагают различные способы более эффективного использования возможностей LLM.

**Источник:** научная публикация, посвящённая исследованиям в области промпт-инжиниринга и работы с большими языковыми моделями.

In [46]:
# Продолжение диалога
result = paper_agent("Расскажи подробнее про Chain-of-Thought")
printx(result.output_text)

**Chain-of-Thought (Цепочка мыслей)** — это техника в промпт-инжиниринге, которая направлена на улучшение способности больших языковых моделей (LLM) решать сложные задачи. Основная идея заключается в том, чтобы заставить модель генерировать серию промежуточных шагов рассуждения, а не сразу выдавать конечный ответ. Это позволяет модели лучше справляться с задачами, требующими сложного логического или математического рассуждения.

**Как работает Chain-of-Thought:**

1. **Генерация промежуточных шагов:** модель создаёт последовательность мыслей или рассуждений, которые ведут к финальному ответу. Это похоже на то, как человек решает задачу шаг за шагом, записывая промежуточные результаты.

2. **Улучшение качества ответов:** разбивая задачу на более мелкие части, модель может лучше понять контекст и требования задачи, что ведёт к более точным и обоснованным ответам.

3. **Применение в различных областях:** техника полезна в широком спектре задач, включая математические и логические головоломки, интерпретацию текста, решение задач, требующих понимания причинно-следственных связей.

**Преимущества:**

- **Повышение точности:** за счёт детального анализа промежуточных шагов модель может избежать ошибок, которые могли бы возникнуть при попытке выдать сразу финальный ответ.
- **Улучшение прозрачности:** цепочка мыслей делает процесс принятия решений моделью более прозрачным и понятным для человека, поскольку видно, как модель пришла к тому или иному выводу.
- **Укрепление способности к сложным рассуждениям:** регулярная практика с использованием Chain-of-Thought может помочь моделям развивать навыки более сложного и структурированного мышления.

**Примеры использования:**

- **Математические задачи:** модель может последовательно выполнять арифметические операции, объясняя каждый шаг.
- **Логические головоломки:** модель может перечислять возможные варианты, исключать неподходящие и приходить к правильному решению.
- **Текстовые задачи:** при анализе текста модель может сначала выделять ключевые факты, затем устанавливать связи между ними и на основе этого делать выводы.

**Источник:** научная публикация, посвящённая исследованиям в области промпт-инжиниринга и работы с большими языковыми моделями.

---

## Часть 6: Добавление новых файлов

Попробуем обернуть нашего агента в диалогового бота, который сможет использоваться как для ответа на вопросы по статьям, так и для добавления новых статей в хранилище. Для этого определим функции, которые будут добавлять файлы или потоки байтов в хранилище.

In [47]:
from io import BytesIO

def add_file_handle(file, filename: str) -> str:
    """
    Добавляет файлоподобный объект в базу знаний.
    
    Args:
        file: Файлоподобный объект (BytesIO и т.д.)
        filename: Имя файла
    
    Returns:
        ID загруженного файла
    """
    print(f"⬆️  Загружаем: {filename}")
    
    uploaded = client.files.create(
        file=(filename, file),
        purpose="assistants"
    )
    
    client.vector_stores.files.create(
        vector_store_id=vector_store.id,
        file_id=uploaded.id,
        chunking_strategy={
            "type": "static",
            "static": {
                "max_chunk_size_tokens": 800,
                "chunk_overlap_tokens": 100
            }
        }
    )
    
    print(f"✅ Файл добавлен: {uploaded.id}")
    return uploaded.id


def add_file(filepath: str) -> str:
    """Загружает файл по пути в базу знаний."""
    print(f"⬆️  Загружаем: {os.path.basename(filepath)}")
    with open(filepath, "rb") as f:
        return add_file_handle(f)


def add_content(content: str) -> str:
    """Добавляет текстовый контент в базу знаний."""
    return add_file_handle(BytesIO(content.encode("utf-8")))


print("✅ Функции добавления файлов определены")

✅ Функции добавления файлов определены


---

## Часть 7: Telegram-бот для работы с базой знаний

Создадим Telegram-бота для работы с научными статьями, который будет уметь:
- Отвечать на вопросы из базы знаний статей
- Загружать прикреплённые файлы в базу статей. Файлы в формате PDF или TXT

Для создания телеграм-бота будем использовать библиотеку [pyTelegramBotAPI](https://pypi.org/project/pyTelegramBotAPI/), также известную как **telebot**. 

> Для запуска бота внутри Jupyter Notebook будем использовать режим polling. Делать так в production environment, конечно же, не нужно. Это всё носит демонстрационный характер. 

Для запуска бота потребуется:

* Создать бота в Telegram с помощью беседы с [@BotFather](https://t.me/botfather) (команда `/newbot`)
* Получить токен
* Добавить его в файл `.env` в текущей директори как `tg_token=YOUR_TOKEN` (либо можете просто установить переменную `tg_token` ниже на соответствующее значение, но постарайтесь, чтобы токен не утёк к вместе с кодом к вашим друзьям)

In [48]:
import telebot

# Получаем токен бота
load_dotenv()
tg_token = os.environ.get("tg_token")

if not tg_token:
    print("⚠️  Токен Telegram не найден в .env (tg_token)")
    print("   Добавьте: tg_token=YOUR_BOT_TOKEN")
else:
    bot = telebot.TeleBot(tg_token)
    print("✅ Telegram-бот инициализирован")

✅ Telegram-бот инициализирован


Ячейки ниже имеет смысл выполнять только в том случае, если вы получили сообщение выше "телеграм-бот успешно инициализирован".

Мы создаём три обработчика команд:
* `start`, который реагирует на команды `/start` или `/help`
* `handle_document`, который отвечает за отправку боту документа. В этом случае вызывается функция `add_file_handle` для добавления документа в файловое хранилище
* `handle_message`, который обрабатывает все остальные сообщения пользователя. Они воспринимаются как вопрос боту, и перенаправляются нашему ранее определённому агенту

In [49]:
@bot.message_handler(commands=["start", "help"])
def start(message):
    help_text = (
        "🤖 **Научный ассистент**\n\n"
        "Я могу:\n"
        "• Отвечать на вопросы по научным статьям\n"
        "• Загружать новые документы (отправьте PDF или TXT)\n\n"
        "Просто напишите вопрос!"
    )
    bot.send_message(message.chat.id, help_text, parse_mode="Markdown")


@bot.message_handler(content_types=["document"])
def handle_document(message):
    """Обработка загруженных файлов."""
    doc = message.document
    filename = doc.file_name
    
    if not (filename.endswith(".pdf") or filename.endswith(".txt")):
        bot.send_message(
            message.chat.id, 
            "⚠️ Поддерживаются только PDF и TXT файлы"
        )
        return
    
    bot.send_message(message.chat.id, f"📥 Загружаю файл: {filename}...")
    
    try:
        file_info = bot.get_file(doc.file_id)
        downloaded = bot.download_file(file_info.file_path)
        
        file_id = add_file_handle(BytesIO(downloaded), filename)
        
        bot.send_message(
            message.chat.id, 
            f"✅ Файл добавлен в базу знаний!\nID: `{file_id}`",
            parse_mode="Markdown"
        )
    except Exception as e:
        bot.send_message(message.chat.id, f"❌ Ошибка: {e}")    

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    """Обработка текстовых сообщений."""
    session_id = str(message.chat.id)
    user_text = message.text
    
    print(f"💬 [{session_id}]: {user_text[:50]}...")
    
    try:
        # Получаем ответ от агента
        result = paper_agent(user_text, session_id=session_id)
        
        # Отправляем ответ
        response = result.output_text or "Не удалось получить ответ"
        
        # Telegram ограничивает длину сообщения
        if len(response) > 4000:
            response = response[:4000] + "..."
        
        bot.send_message(message.chat.id, response)
    except Exception as e:
        bot.send_message(message.chat.id, f"❌ Ошибка: {e}")

print("✅ Обработчики бота настроены")

✅ Обработчики бота настроены


Теперь для запуска бота мы просто запускаем функцию `polling`, которая продолжает работу бесконечно, пока мы её не остановим. Всё время, пока она работает, мы можем взаимодействовать с ботом через телеграм.

In [50]:
print("🚀 Бот запущен! Для остановки: Kernel -> Interrupt")
print("📱 Откройте бота в Telegram и отправьте /start")
bot.polling(none_stop=True)

🚀 Бот запущен! Для остановки: Kernel -> Interrupt
📱 Откройте бота в Telegram и отправьте /start
💬 [217687255]: расскажи про attention...
⬆️  Загружаем: ticket_11615675.pdf
✅ Файл добавлен: fvtnt7k9ajfv7m5pm9ir
💬 [217687255]: нужно ли благодарить нейросети?...
💬 [217687255]: do I need to thank during prompting?...


---

## Часть 8: Очистка ресурсов

В ходе работы мы насоздавали в облаке ресурсов:
* Загруженные файлы
* Поисковые индексы

Удалим все эти ресурсы:

In [ ]:
print(f"🗑️  Удаляем индекс: {vector_store.id}")
client.vector_stores.delete(vector_store_id=vector_store.id)

for f in uploaded_files:
    print(f"🗑️  Удаляем файл: {f.id}")
    client.files.delete(file_id=f.id)

print("✅ Очистка завершена")

🗑️  Удаляем индекс: fvti6gqjnvsmoslkt76b
🗑️  Удаляем файл: fvt3l7a8aq53lpr55f4r
🗑️  Удаляем файл: fvt85d4bbhuc7h5hp8et
🗑️  Удаляем файл: fvt34ibgg34a40atbqld


---

## Заключение

Что мы сделали:

1. Загрузили научные статьи в Yandex Cloud
2. Создали векторное хранилище для семантического поиска
3. Настроили агента с инструментом `file_search`
4. Интегрировали Telegram-бота для загрузки файлов и вопросов

Файловый поиск хорошо использовать для множества задач:
- Корпоративная база знаний
- Техническая документация
- Научные публикации
- FAQ и справочники

Во многих случаях, когда заказчики говорят "давайте дообучим нейросеть на нашу предметную область", речь идёт именно о добавлении файлового поиска, а не о до-обучении сети. Именно файловый поиск позволяет добавить агенту новых знаний.